In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import fasttext
from convokit import Corpus, download
from xgboost.sklearn import XGBRegressor
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')

In [3]:
# corpus_oanc = PlaintextCorpusReader('./OANC-GrAF', '.*\.txt')

In [4]:
# full_m1 = fasttext.train_unsupervised('corpus_general_plain.txt', model='cbow')
# full_m1.save_model('full_m1.bin')

In [5]:
# full_m2 = fasttext.train_unsupervised('corpus_general_plain.txt', model='skipgram')
# full_m2.save_model('full_m2.bin')

In [6]:
# sample_m1 = fasttext.train_unsupervised('corpus_specialized_plain.txt', model='cbow')
# sample_m1.save_model('sample_m1.bin')

In [7]:
# sample_m2 = fasttext.train_unsupervised('corpus_specialized_plain.txt', model='skipgram')
# sample_m2.save_model('sample_m2.bin')

In [8]:
data = pd.read_csv('corpus_specialized_annotated.csv')
data['text'] = data['text'].astype(str)

In [9]:
def get_vectorizer(model):
    def get_text_repr(text):
        words = nltk.word_tokenize(text)
        if words:
            return np.stack([model.get_word_vector(word) for word in words]).mean(0)
        else:
            return np.zeros(100)
    return get_text_repr

def prepare_data(data, model):
    vectorizer = get_vectorizer(model)
    X = np.stack(data['text'].apply(vectorizer))
    y = np.array(data['meta.score'])
    mask = np.random.rand(len(X)) < 0.8
    X_train = X[mask]
    X_test = X[~mask]
    y_train = y[mask]
    y_test = y[~mask]
    return X_train, y_train, X_test, y_test

In [10]:
for model_path in ['full_m1.bin', 'full_m2.bin', 'sample_m1.bin', 'sample_m2.bin']:
    vectorizer = fasttext.FastText.load_model(model_path)
    X_train, y_train, X_test, y_test = prepare_data(data, vectorizer)
    regressor = XGBRegressor(tree_method='gpu_hist')
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    print(model_path)
    print('\tmse:\t', mean_squared_error(y_test, y_pred))
    print('\tr2:\t', r2_score(y_test, y_pred))

C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\apply.py:1043: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return self.apply_standard()


full_m1.bin
	mse:	 1489.5028610774439
	r2:	 -0.16548617696787327


C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\apply.py:1043: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return self.apply_standard()


full_m2.bin
	mse:	 2618.810910345214
	r2:	 -0.07255739762355584


C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\apply.py:1043: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return self.apply_standard()


sample_m1.bin
	mse:	 2377.7832091785417
	r2:	 -0.07574189896015726


C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\apply.py:1043: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return self.apply_standard()


sample_m2.bin
	mse:	 1484.007337722252
	r2:	 -0.17366357840864977


In [18]:
y_train.shape

(463523,)